In [21]:
import torch

In [22]:
with open('Iran.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
vocabulary_size = len(chars)

In [23]:
def encode(string) -> list:
    return [string_to_int[c] for c in string]

def decode(l) -> str:
    return ''.join([int_to_string[s] for s in l])

In [24]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([106,  48,  65,  62,   1,  44,  75,  72,  67,  62,  60,  77,   1,  35,
         78,  77,  62,  71,  59,  62,  75,  64,   1,  62,  30,  72,  72,  68,
          1,  72,  63,   1,  48,  65,  62,   1,  44,  62,  75,  76,  66,  58,
         71,   1,  40,  66,  77,  62,  75,  58,  77,  78,  75,  62,  11,   1,
         31,  72,  70,  73,  75,  66,  76,  66,  71,  64,   1,  48,  65,  62,
          1,  47,  65,  58,  65,   1,  42,  58,  70,  62,  65,  11,   1,  48,
         65,  62,   1,  46,  78,  59,  58,  66,  82,  58,  77,  11,   1,  48,
         65,  62])


In [25]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]